## Garbage Classifier 🗑️ 🚮 🚯

### ⚙️ Setup

In [1]:
# Import libraries
import os
from dotenv import load_dotenv
import torch

In [2]:
# Reload for caching
from importlib import reload
import src.base_dataset as dataset
reload(dataset)
import src.dataset_loader as dataset_loader
reload(dataset_loader)
import src.garbage_model as garbage_model
reload(garbage_model)
import src.my_transforms as my_transforms
reload(my_transforms)
import src.train_utils as train_utils
reload(train_utils)

from src.base_dataset import BaseDataset
from src.garbage_model import GarbageModel
from src.dataset_loader import list_data_and_prepare_labels, split_data
from src.my_transforms import torch_vision_transform, torch_vision_transform_test
from src.train_utils import train_validate

In [3]:
# get file directory
curr_dir = os.getcwd()

# load dataset_path
dotenv_path = os.path.join(curr_dir, ".env")
load_dotenv(dotenv_path)

True

In [4]:
# Constants
EPOCHS = 12
LEARNING_RATE = 2e-4
TEST_SPLIT = 0.2
VAL_SPLIT = 0.2
BATCH_SIZE = 8
NUM_CLASSES = 4
INPUT_SHAPE = (3, 224, 224)
INPUT_SIZE = (1, 3, 256, 256)

In [5]:
# Variables
dataset_path = os.getenv("DATASET_LOCAL_PATH")
normalized_path = dataset_path
best_model_path = os.getenv("MODEL_PATH")

### ↻ Data Loader

In [6]:
# get dataset
images_path = normalized_path + "/**/*.png"
images, labels_int, classes = list_data_and_prepare_labels(images_path)

In [7]:
# split dataset
all_dataset = split_data(images, labels_int, VAL_SPLIT, TEST_SPLIT)
train_set = all_dataset["Train"]
val_set = all_dataset["Validation"]
test_set = all_dataset["Validation"]

In [8]:
# Get the dataset
train_dataset = BaseDataset(train_set, transform=torch_vision_transform)
val_dataset = BaseDataset(val_set, transform=torch_vision_transform)
test_dataset = BaseDataset(test_set,transform= torch_vision_transform_test)

In [9]:
# Get data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

### 🏃‍♂️ Train

In [10]:
# load model
net_18 = GarbageModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES, transfer=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net_18.to(device)

c:\Users\chris\miniconda3\envs\calgary_di\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\chris\miniconda3\envs\calgary_di\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GarbageModel(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [11]:
train_validate(net_18, train_loader, val_loader, EPOCHS, LEARNING_RATE, best_model_path, device)

Epoch 1, Train loss: 1.427 Val loss: 1.060
Saving model
Epoch 2, Train loss: 1.096 Val loss: 0.949
Saving model
Epoch 3, Train loss: 0.919 Val loss: 1.053
Epoch 4, Train loss: 0.948 Val loss: 1.043
Epoch 5, Train loss: 0.739 Val loss: 0.807
Saving model
Epoch 6, Train loss: 0.829 Val loss: 0.827
Epoch 7, Train loss: 0.699 Val loss: 0.911
Epoch 8, Train loss: 0.607 Val loss: 0.698
Saving model
Epoch 9, Train loss: 0.634 Val loss: 0.724
Epoch 10, Train loss: 0.742 Val loss: 0.702
Epoch 11, Train loss: 0.738 Val loss: 0.757
Epoch 12, Train loss: 0.589 Val loss: 0.700
Finished Training
